In [1]:
%load_ext autoreload
%autoreload 2

True

In [5]:
import pandas as pd 
import yake
import gensim.downloader
import gensim
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm
import pandas as pd

In [6]:
cosine = lambda a,b : np.dot(a,b)/(norm(a)*norm(b))
euclidian = lambda a,b : norm(a-b)

In [7]:
def cosine(a,b):
    d = np.dot(a,b)
    na = norm(a)
    nb = norm(b)
    nanb = na*nb
    return d / nanb

In [8]:
# vect = gensim.downloader.load("fasttext-wiki-news-subwords-300")

vect = gensim.models.KeyedVectors.load_word2vec_format(r"C:\Users\romaf\gensim-data\wiki-2017-gensim\model.bin", binary=True)

In [62]:
kwords = yake.KeywordExtractor()

In [63]:
df = pd.read_csv('data/submission.csv',index_col=['#'])

In [64]:
def get_vectors(data):
    vectorised_data = []
    for sentence in data:
        sentence = sentence.replace("-"," ")
        words = sentence.split()
        sentence_vector = np.zeros([300])
        for word in words:
            try:
                sentence_vector = sentence_vector + vect[word.lower()]
            except Exception:
                sentence_vector = None
                break
        vectorised_data.append(sentence_vector) 
    return vectorised_data

In [65]:
def get_sum_vec(data):
    sum_vec1 = np.zeros([300])
    for vec in data:
        if vec is not None:
            sum_vec1 = sum_vec1 + vec
    return sum_vec1

In [87]:
def get_weight_sum_vec(data):
    sum_vec1 = np.zeros([300])
    for ind,vec in enumerate(data):
        if vec is not None:
            sum_vec1 = sum_vec1 + (vec * (1 - 0.01 * ind)) 
    return sum_vec1

In [88]:
def get_vec_rpr(data):
    title,keys,abstract = data

    kw_from_abstract = [words for words,val in kwords.extract_keywords(abstract)]

    kw_from_keys = [word for word in keys.split('\n')]

    vec_abstract = get_vectors(kw_from_abstract)

    sum_vec_abstract = get_weight_sum_vec(vec_abstract)

    vec_keys = get_vectors(kw_from_keys)

    sum_vec_keys = get_weight_sum_vec(vec_keys)

    return sum_vec_abstract,sum_vec_keys,sum_vec_abstract+sum_vec_keys

In [67]:
def get_vec_rpr_kw_only(data):
    keys = data

    kw_from_keys = data

    vec_keys = get_vectors(kw_from_keys)

    sum_vec_keys = get_sum_vec(vec_keys)

    return sum_vec_keys

In [68]:
df2 = df[["title",'keywords','abstract']]

In [70]:
data = []
for ind,row in df2.iterrows():
    data.append([row['title'],row['keywords'],row['abstract']])

In [71]:
data[64]

['Autoencoding Language Model Based Ensemble Learning for Commonsense Validation and Explanation',
 'Autoencoding Language Model\nSiamese Neural Network\nEnsemble Learning\nCommonsense Validation\nCommonsense Explanation',
 'An ultimate goal of artificial intelligence is to build computer systems that can understand human languages. Understanding commonsense knowledge about the world expressed in text is one of the foundational and challenging problems to create such intelligent systems. As a step towards this goal, we present in this paper ALMEn, an Autoencoding Language Model based Ensemble learning method for commonsense validation and explanation. By ensembling several advanced pre-trained language models including RoBERTa, DeBERTa, and ELECTRA with Siamese neural networks, our method can distinguish\r\nnatural language statements that are against commonsense (validation subtask) and correctly identify the reason for making against commonsense (explanation selection subtask). Exper

In [89]:
ab,kw,abkw = get_vec_rpr(data[1])
a,k,ak = get_vec_rpr(data[64])
ab = ab+a
kw = kw +k
abkw = abkw + ak

In [90]:
res = []
for data_val in tqdm(data):
    title,keys,abstract = data_val
    a,k,ak =get_vec_rpr(data_val)
    res.append({"title":title,"keys":keys,"abstract":abstract,"a":cosine(ab,a),'k':cosine(kw,k),'ak':cosine(abkw,ak)})


 92%|█████████▏| 316/344 [00:41<00:04,  6.91it/s]C:\Users\romaf\AppData\Local\Temp\ipykernel_14296\3083299470.py:1: RuntimeWarning: invalid value encountered in double_scalars
  cosine = lambda a,b : np.dot(a,b)/(norm(a)*norm(b))
100%|██████████| 344/344 [00:45<00:00,  7.52it/s]


In [91]:
df = pd.DataFrame(res)

In [92]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [93]:
df['mean_ak'] = ((df['a'] + df['k']) /2)

In [97]:
df.sort_values(by=['ak'],ascending=False)[['title','a','k','ak','mean_ak']].head(100)

,title,a,k,ak,mean_ak
1,XLMRQA: Open-Domain Question Answering on Vietnamese Wikipedia-based Textual Knowledge Source,0.982956,0.944438,0.983605,0.963697
64,Autoencoding Language Model Based Ensemble Learning for Commonsense Validation and Explanation,0.963838,0.892910,0.965764,0.928374
317,OAK4XAI: Model towards Multi-Layer eXplainable Artificial Intelligence for Digital Agriculture,0.897037,0.725682,0.915372,0.811359
311,Error Investigation of Pre-trained BERTology Models on Vietnamese Natural Language Inference,0.879471,0.831736,0.891619,0.855603
38,Corpus based approach to semantic analysis Uzbek text,0.883444,0.756664,0.891000,0.820054
253,Improving Topic Quality with Interactive Beta-Liouville Mixture Allocation Model,0.860567,0.803747,0.884422,0.832157
145,End to End Neural Network for Tweet Text Prediction,0.884704,0.687556,0.876731,0.786130
62,Abstractive Text Summarization Utilising Pretrained Language Models,0.854707,0.743920,0.876663,0.799313
287,Speech Emotion Classification - A Survey of the State-of-the-Art,0.876870,0.699556,0.874467,0.788213
165,Performance Analysis of Digit Recognizer usingVarious Machine Learning Algorithms,0.861795,0.799101,0.874252,0.830448


In [72]:
df.iloc[32]

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Explainable Machine Learning for Sequences of Demographic Statuses
keys                                              

In [73]:
data1 = data[8]
data2 = data[40]

In [74]:
data1_kw = kwords.extract_keywords(data1[2])
data1_kw = [words for words,val in data1_kw]

In [75]:
data2_kw = kwords.extract_keywords(data2[2])
data2_kw = [words for words,val in data2_kw]

In [76]:
data1_kw

['communicate online',
 'Danmaku',
 'sentiment',
 'Network TBAN',
 'danmaku sentiment analysis',
 'sentiment analysis',
 'time',
 'improved danmaku sentiment',
 'time features',
 'video content',
 'WiththedevelopmentofInternet,danmakuoffersanewway for users',
 'requires external informa',
 'users to communicate',
 'danmaku sentiment',
 'carries limited information',
 'features',
 'WiththedevelopmentofInternet,danmakuoffersanewway',
 'online',
 'sentiment analysis method',
 'communicate']

In [77]:
data2_kw

['Social media',
 'social media posts',
 'interest areas',
 'accessible and effective',
 'individuals to offer',
 'offer thoughts',
 'range of interest',
 'sentiment analysis',
 'social media platforms',
 'sentiment analysis methods',
 'sentiment',
 'Natural Language Processing',
 'Social',
 'media',
 'analysis',
 'Naive Bayes Classifiers',
 'large volume',
 'effective platform',
 'wide range',
 'Word Sense Disambiguation']

In [78]:
data1_kw_v = get_vectors(data1_kw)

In [79]:
data2_kw_v = get_vectors(data2_kw)

In [80]:
sum_vec1 = np.zeros([300])
for vec in data1_kw_v:
    if vec is not None:
        sum_vec1 = sum_vec1 + vec

In [81]:
sum_vec2 = np.zeros([300])
for vec in data2_kw_v:
    if vec is not None:
        sum_vec2 = sum_vec2 + vec

In [82]:
cosine(sum_vec1,sum_vec2)

0.8559171651111073

In [83]:
data1_l_kw = []
for word in data1[1].split('\n'):
    data1_l_kw.append(word)

In [84]:
data2_l_kw = []
for word in data2[1].split('\n'):
    data2_l_kw.append(word)

In [85]:
data1_l_kw

['sentiment analysis', 'danmaku', 'RNN', 'attention']

In [86]:
data1_l_kw_v = get_vectors(data1_l_kw)
data2_l_kw_v = get_vectors(data2_l_kw)

In [87]:
sum_vec1_l = np.zeros([300])
for vec in data1_l_kw_v:
    if vec is not None:
        sum_vec1_l = sum_vec1_l + vec

sum_vec2_l = np.zeros([300])
for vec in data2_l_kw_v:
    if vec is not None:
        sum_vec2_l = sum_vec2_l + vec

In [88]:
cosine(sum_vec1_l+sum_vec1,sum_vec2_l+sum_vec2)

0.8763649527072961